In [ ]:
import pandas as pd
import numpy as np
import ast

fixedDf = pd.read_csv('data/fixedData.csv')
fixedDf.head()

# Get Data Reviewer

In [121]:
fixedDf = fixedDf.rename(columns={'Unnamed: 0': 'place_id'})
fixedDf.place_id.nunique()
fixedDf.to_csv('data/fixedData.csv', index=False)

## Tambah Kolom Place Id

In [ ]:
reviewerDf = fixedDf[['place_id','reviewer_name', 'rating_review', 'review_text']]
reviewerDf.info()

## Memisahkan Data List dalam Bentuk Per Baris Data

### Mengisi Nilai Null dengan Anonim

In [123]:
# Langkah 1: Mengisi nilai null dengan string '["anonim"]'
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].fillna('["unknown"]')
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].fillna('["0.0"]')
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].fillna('["unknown"]')

### Preprocess Bentuk List

In [124]:
# Fungsi aman untuk menggunakan ast.literal_eval
def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)  # Ubah hanya jika val adalah string
        return val
    except (ValueError, SyntaxError):
        return val  # Kembalikan val asli jika tidak bisa diuba

In [125]:
# Langkah 2: Mengonversi string yang terlihat seperti list menjadi list Python menggunakan ast.literal_eval secara aman
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].apply(safe_literal_eval)
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].apply(safe_literal_eval)
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].apply(safe_literal_eval)

### Pemecahan List Jadi Per Baris

In [126]:
# Langkah 3: Menentukan panjang maksimum list dalam setiap kolom
def get_max_len(col):
    return col.apply(lambda x: len(x) if isinstance(x, list) else 0).max()

max_len_reviewer = get_max_len(reviewerDf['reviewer_name'])
max_len_rating = get_max_len(reviewerDf['rating_review'])
max_len_review = get_max_len(reviewerDf['review_text'])
max_len = max(max_len_reviewer, max_len_rating, max_len_review)

In [127]:
# Langkah 4: Menyamakan panjang list di semua kolom dengan mengisi NaN pada list yang lebih pendek
def pad_list_with_nan(lst, max_len):
    if isinstance(lst, list):
        return lst + [np.nan] * (max_len - len(lst))
    else:
        return [np.nan] * max_len  # Jika tidak ada list, buat list dengan NaN
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].apply(lambda x: pad_list_with_nan(x, max_len))
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].apply(lambda x: pad_list_with_nan(x, max_len))
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].apply(lambda x: pad_list_with_nan(x, max_len))

# Langkah 5: Meledakkan (explode) kolom reviewer_name, rating_review, dan review_text
reviewerDf = reviewerDf.explode(['reviewer_name', 'rating_review', 'review_text']).reset_index(drop=True)

### Hapus Data NaN (Dummy)

In [128]:
reviewerDf = reviewerDf.dropna(axis=0)

### Tambah Data User ID

In [ ]:
reviewerDf['user_id'] = pd.factorize(reviewerDf['reviewer_name'])[0] + 1
# Ubah Urutan Kolom Agar Mudah dibaca
reviewerDf = reviewerDf[['place_id', 'user_id', 'reviewer_name', 'rating_review', 'review_text']]
reviewerDf.head(20)

In [130]:
reviewerDf.to_csv('data/data-user/originalData.csv', index=False)

# Exploratory Data Analysis (EDA)

Pertanyaan:
1. Berapa user id yang lebih dari 1 nilai unik yang ada di dalam data?
2. place_id apa saja yang direview oleh user id yang tersaring di pertanyaan 1?

## Jawab No 1

In [ ]:
# Menghitung jumlah kemunculan setiap user_id yang muncul lebih dari 1 kali
user_id_counts = reviewerDf['user_id'].value_counts()
filtered_user_ids = user_id_counts[user_id_counts > 1]

# Visualisasikan hasil group user_id yang muncul lebih dari 1 kali dengan countnya
cntByUsrID = pd.DataFrame({'user_id': filtered_user_ids.index, 'count': filtered_user_ids.values})
# Mengubah hasil menjadi format yang lebih mudah dibaca
cntByUsrID = cntByUsrID.groupby('count')['user_id'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
cntByUsrID

In [ ]:
reviewerDf.loc[reviewerDf['user_id'] == 49, 'reviewer_name']

In [133]:
cntByUsrID.to_csv('data/data-user/countUserPerReview.csv', index=False)

## Jawab No 2

In [ ]:
# All
# Memodifikasi kode untuk memisahkan review_text dengan newline (\n) tanpa menggunakan unique
grpPlaceAll = reviewerDf.groupby('user_id').apply(
    lambda x: pd.DataFrame({
        'place_ids': [', '.join(x['place_id'].astype(str).unique())],
        'ratings': [', '.join(x['rating_review'].astype(str))],
        'reviews': ['\n'.join(x['review_text'])]  # Memisahkan review dengan \n
    })
).reset_index(level=1, drop=True).reset_index()

grpPlaceAll

In [ ]:
# Sebagian
# Filter dataframe berdasarkan user_id yang muncul lebih dari 1 kali
filtered_df = reviewerDf[reviewerDf['user_id'].isin(filtered_user_ids.index)]

# Modifikasi kode untuk mengelompokkan data berdasarkan user_id yang sudah difilter
grpUserFiltered = filtered_df.groupby('user_id').apply(
    lambda x: pd.DataFrame({
        'place_ids': [', '.join(x['place_id'].astype(str).unique())],
        'ratings': [', '.join(x['rating_review'].astype(str))],
        'reviews': ['\n'.join(x['review_text'])]  # Memisahkan review dengan \n
    })
).reset_index(level=1, drop=True).reset_index()
grpUserFiltered

In [136]:
grpUserFiltered.to_csv('data/data-user/countUserPerUniqueReview.csv', index=False)

In [ ]:
# Ambil data reviews dari user_id 16
reviews = grpUserFiltered.loc[grpUserFiltered['user_id'] == 16, 'reviews'].values

# Jika reviews ditemukan, pisahkan berdasarkan '\n' dan print setiap review
if reviews:
    for review in reviews[0].split('\n'):
        print(review)
else:
    print("No reviews found for user_id 16.")